In [ ]:
# Procesa el dataframe para el machine learning.

import pandas as pd

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/" 
path_data = path_drive + "data/data_"
path_socio = path_drive + "Socioeconomicos/"
csv = ".csv"
socio = [ "Electrico_residencial_2010_2017.csv",
    "Datos demográficos.csv", "GLP_2010_2016.csv" ]

data = pd.read_csv(path_data + str(0) + csv)

In [ ]:
# Agrega las variables procesadas para cada municipio.
for i in range(1, 10):
    data_i = pd.read_csv(path_data + str(i) + csv)
    data.iloc[:, i + 10] = data_i.iloc[:,i + 3]

In [ ]:
# Agrega el preico de gas LP.
df_socio = pd.read_csv(path_socio + socio[2])
for j in range(2010, 2017):
    for i in range(1, 33):
        data.loc[ (data["Año"] == j) & (data["CVE_ENT"] == i),
        "$GLP" ] = df_socio.loc[
        df_socio["CVE_ENT"] == i, str(j)].values[0]

# Agrega la población y el PIB.    
df_socio = pd.read_csv(path_socio + socio[1])
for j in range(2010, 2017):
    for i in data["CVE_INEGI"].unique():
        for k in ["Población", "PIB"]:
            data.loc[ (data["Año"] == j) & (data["CVE_INEGI"] == i),
                k ] = df_socio.loc[df_socio["CVE_INEGI"] == i,
                k + " " + str(j)].values[0]

# Agrega el consumo y la cantidad de usuarios.
df_socio = pd.read_csv(path_socio + socio[0])
for j in range(2010, 2017):
    for i in data["CVE_INEGI"].unique():
        for k in ["Consumo", "Usuarios"]:
            for l in ["1*", "DAC"]:
                data.loc[ (data["Año"] == j) & (data["CVE_INEGI"] == i),
                    k + "_" + l ] = df_socio.loc[(df_socio["CVE_INEGI"] == i)
                    & df_socio["Tarifa"].str.contains(l, regex = True),
                    k + " " + str(j)].values[0]

# PIB per cápita y densidad de población.
data["PCI"] = data["PIB"] / data["Población"]
data["Densidad_población"] = data["Población"] / data["Area"]

In [ ]:
# Escogemos las variables relevantes.
data_proc = data.drop(["Area", "CVE_ENT", "CVE_MUN",
  "CVE_INEGI", "T_max", "T_min", "HDD_mean",
  "HDD_p10",  "CDD_p90", "$luz"], axis = 1)

data_proc.to_csv(path_data[:-1] + csv, index = False) )

# Correlación entre variables.
data_proc.corr()